### Load Packages

In [1]:
import pathlib
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
%matplotlib inline
from IPython import display
import pandas as pd
import sys, subprocess
from PIL import Image
import pickle
from armcodlib import *


/usr/local/lib/python2.7/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)
Using TensorFlow backend.


In [2]:
print(tf.__version__)

2.0.0-dev20190519


### Create arm positions dataset

In [16]:
NB_POSTURE = 600
NB_COMMAND = 10
NB_DATA = NB_POSTURE*NB_COMMAND
BATCH_SIZE = 100
TEST_BUF = 1000
DIMS = (128,128,2)
N_TRAIN_BATCHES =int(NB_DATA/BATCH_SIZE)
N_TEST_BATCHES = int(TEST_BUF/BATCH_SIZE)



In [9]:
train_images_pickle = open("pickles/train_images.pickle", "rb")
test_images_pickle = open("pickles/test_images.pickle", "rb")
train_command_pickle = open("pickles/train_command.pickle", "rb")
test_command_pickle = open("pickles/test_command.pickle", "rb")

train_command = pickle.load(train_command_pickle)
train_images = pickle.load(train_images_pickle)
test_command = pickle.load(test_command_pickle)
test_images = pickle.load(test_images_pickle)

In [10]:
train_images = np.expand_dims(train_images, axis=1)
test_images = np.expand_dims(test_images, axis=1)

In [17]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices((train_images,train_images[:,:,:,:,1:]))
    .shuffle(NB_DATA)
    #.batch(BATCH_SIZE)
)

test_dataset = (
    tf.data.Dataset.from_tensor_slices((test_images, test_images[:,:,:,1:]))
    .shuffle(TEST_BUF)
    #.batch(BATCH_SIZE)
)

### Define the network architecture

In [ ]:
encoder = build_dense_encoder()
decoder = build_dense_decoder()

### Create Model

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-3)

autoencoder = tf.keras.Model(encoder.input, decoder(encoder(encoder.input)), name = "autoencoder")

autoencoder.compile(optimizer = optimizer, 
             loss='mse',
             metrics=['accuracy','kullback_leibler_divergence'])

### Train model

In [ ]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = autoencoder.fit(train_dataset, 
                    epochs = 10,
                    callbacks=[tensorboard_callback])

In [ ]:
example_data = next(iter(train_dataset))

In [ ]:
imshow(autoencoder.predict(example_data)[0,:,:,0])

In [ ]:
imshow(example_data[1][0,:,:,0])

In [ ]:
shape(train_images[:,:,:,:,1:])

### Alternative convolutional model

In [12]:
from armcodlib import *

In [18]:
encoder_conv = build_conv2D_encoder()
decoder_conv = build_conv2D_decoder()


In [19]:
optimizer = tf.keras.optimizers.Adam(1e-3)

autoencoder_conv = tf.keras.Model(encoder_conv.input, decoder_conv(encoder_conv(encoder_conv.input)), name = "autoencoder")

autoencoder_conv.compile(optimizer = optimizer, 
             loss='mse',
             metrics=['accuracy','kullback_leibler_divergence'])

In [20]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = autoencoder_conv.fit(train_dataset, 
                    epochs = 10,
                    callbacks=[tensorboard_callback])

Epoch 1/10


InvalidArgumentError: Input to reshape is a tensor with 128 values, but the requested shape has 4096
	 [[{{node encoder_model_3/reshape_25/Reshape}}]] [Op:__inference_keras_scratch_graph_6232]